In [ ]:
import matplotlib
matplotlib.use('Agg')

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()

import sys
import os
import glob

ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
PASCAL_VOC_DATA = '/gpfs01/bethge/data/PascalVOC'
MASK_RCNN_MODEL_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/Mask_RCNN/'
SIAMESE_MASK_RCNN_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
import utils as siamese_utils
import model as siamese_model
import siamese_mrcnn_models as model_zoo
    
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
exp_dir = ROOT_DIR #os.path.join(ROOT_DIR,'coco_full')

In [ ]:
coco_nopascal_classes = [8,10,11,12,13,14,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80]
coco_pascal_classes = np.array(range(1,81))[np.array([i not in coco_nopascal_classes for i in range(1,81)])]

In [ ]:
class TrainConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 12
    LEARNING_RATE = 0.001
    NUM_CLASSES = 1 + 1
    TARGET_MAX_DIM = 96
    TARGET_MIN_DIM = 75
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    #IMAGE_RESIZE_MODE = 'none'
    TARGET_SHAPE = np.array([TARGET_MAX_DIM, TARGET_MAX_DIM, 3])
    TARGET_PADDING = True
    MAX_TARGET_INSTANCES = 10
    # Reduce model size for prototyping
    BACKBONE = 'resnet50'
    FPN_FEATUREMAPS = 128
    RPN_ANCHOR_STRIDE = 2
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    POST_NMS_ROIS_TRAINING = 250
    POST_NMS_ROIS_INFERENCE = 250
    TRAIN_ROIS_PER_IMAGE = 50
    DETECTION_MAX_INSTANCES = 15
    DETECTION_NMS_THRESHOLD = 0.5
    MAX_GT_INSTANCES = 15
    LOSS_WEIGHTS = {'rpn_class_loss': 2.0, 
                    'rpn_bbox_loss': 0.1, 
                    'mrcnn_class_loss': 2.0, 
                    'mrcnn_bbox_loss': 0.5, 
                    'mrcnn_mask_loss': 1.0}
    NAME = "coco"
    EXPERIMENT = "full"
    # Imagenet checkpoint 
    PRETRAINING = 'imagenet-1k' # or 'imagenet-771'
    if PRETRAINING == 'imagenet-1k':
        IMAGENET_CHECKPOINT = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/logs/imagenet20180511T1119/mask_rcnn_imagenet_0810.h5'
    elif PRETRAINING == 'imagenet-771':
        IMAGENET_CHECKPOINT = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/logs/imagenet20180520T2051/mask_rcnn_imagenet_0784.h5'
    else: 
        print("pretraining must be either 'imagenet-1k' or 'imagenet-771'")
        
config = TrainConfig()
config.display()

In [ ]:
# Load COCO/train dataset
coco_train = siamese_utils.IndexedCocoDataset()
coco_train.set_active_classes(coco_nopascal_classes)
coco_train.load_coco(COCO_DATA, "train", year="2017")
coco_train.prepare()
coco_train.build_indices()
coco_train.ACTIVE_CLASSES = np.array(range(1,81))

# Load COCO/val dataset
coco_val = siamese_utils.IndexedCocoDataset()
coco_val.set_active_classes(coco_nopascal_classes)
coco_val.load_coco(COCO_DATA, "val", year="2017")
coco_val.prepare()
coco_val.build_indices()
coco_val.ACTIVE_CLASSES = np.array(range(1,81))

### Simple

In [ ]:
class SimpleSiameseMaskRCNN(model_zoo.SimpleSiameseMaskRCNN):
    
    def set_log_dir(self, model_path=None):
        """Sets the model log directory and epoch counter.
        model_path: If None, or a format different from what this code uses
            then set a new log directory and start epochs from 0. Otherwise,
            extract the log directory and the epoch counter from the file
            name.
        """
        # Set date and epoch counter as if starting a new model
        self.epoch = 0
        now = datetime.datetime.now()

#         # If we have a model path with date and epochs use them
#         if model_path:
#             # Continue from we left of. Get epoch and date from the file name
#             # A sample model path might look like:
#             # /path/to/logs/coco20171029T2315/mask_rcnn_coco_0001.h5
#             regex = r".*/[\w-]+(\d{4})(\d{2})(\d{2})T(\d{2})(\d{2})/mask\_rcnn\_[\w-]+(\d{4})\.h5"
#             m = re.match(regex, model_path)
#             if m:
#                 now = datetime.datetime(int(m.group(1)), int(m.group(2)), int(m.group(3)),
#                                         int(m.group(4)), int(m.group(5)))
#                 # Epoch number in file is 1-based, and in Keras code it's 0-based.
#                 # So, adjust for that then increment by one to start from the next epoch
#                 self.epoch = int(m.group(6)) - 1 + 1
#                 print('Re-starting from epoch %d' % self.epoch)

        # Directory for training logs
        self.log_dir = os.path.join(self.model_dir, 
                                    "{}_{}".format(self.config.NAME.lower(), 
                                                   self.config.EXPERIMENT.lower()))

        # Create log_dir if not exists
        if not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)

        # Path to save after each epoch. Include placeholders that get filled by Keras.
        self.checkpoint_path = os.path.join(self.log_dir, "siamese_mrcnn_*epoch*.h5")
        self.checkpoint_path = self.checkpoint_path.replace("*epoch*", "{epoch:04d}")

In [ ]:
# Create model object in inference mode.
model = SimpleSiameseMaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

In [ ]:
# Load weights trained on Imagenet
try: 
    os.path.exists(os.path.join(model.log_dir, "siamese_mrcnn_0001.h5"))
    list_of_files = glob.glob(os.path.join(model.log_dir,'*.h5')) # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getmtime)
    epoch_index = int(latest_file[-7:-3])
    print('loading', latest_file, '...')

    # set layers trainable for resnet weight loading
    if epoch_index <= 120:
        model.set_trainable("(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")
    elif epoch_index <= 240:
        model.set_trainable("(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")          
    elif epoch_index <= 480:
        model.set_trainable("(res2.*)|(bn2.*)|(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")
    else:
        model.set_trainable(".*")
    # load weights            
    model.load_weights(latest_file, by_name=True)
    model.epoch = epoch_index
except:
    print('initializing from imagenet weights ...')
    model.load_weights(config.IMAGENET_CHECKPOINT, by_name=True)
    model.set_log_dir()




In [ ]:
# file = open(os.path.join(exp_dir, "started.txt"),"w")
# file.close()
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE, epochs=120, layers="heads")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE, epochs=240, layers="4+")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE, epochs=360, layers="2+")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE, epochs=480, layers="all")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE/10, epochs=510, layers="all")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE/100, epochs=540, layers="all")
file = open(os.path.join(exp_dir, "finished.txt"),"w")
file.close()
print("Done")